# ModelV2
## Object detection non-overlapping cards

In [4]:
!pip -q install ultralytics wandb


In [5]:
from ultralytics import YOLO
from jassair.utils.paths import get_dataset_path, Datasets
import wandb
import time
import torch
import random
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

### Global variable definition

In [6]:
DATA_DIR = get_dataset_path(Datasets.SWISS)
BATCH_SIZE = 32
NUM_CLASSES = 36

In [7]:
torch.cuda.is_available()

True

Enable W&B logging for Ultralytics in CLI

`yolo settings wandb=True`

Disable Weights & Biases logging

`yolo settings wandb=False`

In [8]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: aarfu (aarfu-hochschule-luzern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Model
We can use a singel YOLO model for both, object detection and classification tasks.

In [9]:
MODEL = YOLO("yolo11n.pt") # YOLOv11 nano for quick testing. Later we choose the larger model for better accuracy

100%|██████████| 5.35M/5.35M [00:00<00:00, 56.5MB/s]


## Train
imgsz: yolo resizes the input images to a square image. Since jasscards are not square, yolo resizes the long side to 640 and then scales the shorter side proportionally.

In [10]:
# Hyperparameters
# epochs=50
# imgsz=640 # 640 (default) – Balanced performance; 512 – Faster, lower memory usage, 1280+ – Best accuracy for small details but slower.
# iterations=100 # Number of tuning cycles (higher = better tuning). The model tries different values for lr, momentum, weight decay, etc.

HYPERPARAMETERS = {
    "lr0": 1e-3,        # renamed from "lr" to "lr0"
    "weight_decay": 1e-5,  # renamed from "wd"
    "imgsz": 640,
    "epochs": 100,
}

In [11]:
RUN_NAME = f"{time.strftime('%y%m%d-%H%M%S')}_{MODEL._get_name()}_lr{HYPERPARAMETERS['lr0']}"

In [12]:
wandb.init(
    entity="jassair",
    project="ModelV2",  # Specify project
    name=RUN_NAME, # Run name
    config=HYPERPARAMETERS,
)

When you choose iterations, the model tries to tune itself. Or you can tune them manually like below

In [13]:
# Model with all tunable hyperparameters
MODEL.train(
    data="/exchange/dspro2/jassair/data.yaml",
    device=0,
    batch=BATCH_SIZE,
    optimizer="Adam",
    momentum=0.937,
    degrees=10,
    translate=0.1,
    scale=0.5,
    shear=2.0,
    perspective=0.001,
    flipud=0.1,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    project="jassair",
    name="test_run_1",
    patience=20,
    **HYPERPARAMETERS
)


Ultralytics 8.3.98 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA A16, 14891MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/exchange/dspro2/jassair/data.yaml, epochs=100, time=None, patience=20, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=jassair, name=test_run_14, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

train: Scanning /exchange/dspro2/jassair/data_v2.1/train/labels.cache... 9994 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9994/9994 [00:00<?, ?it/s]
val: Scanning /exchange/dspro2/jassair/data_v2.1/valid/labels.cache... 2000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]


Plotting labels to jassair/test_run_14/labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=1e-05), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to jassair/test_run_14
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100       4.4G      1.159      2.178      1.175         71        640: 100%|██████████| 313/313 [02:24<00:00,  2.16it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:14<00:00,  2.20it/s]


                   all       2000       9002      0.665      0.684      0.791      0.445

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


80 epochs completed in 3.450 hours.
Optimizer stripped from jassair/test_run_14/weights/last.pt, 5.5MB
Optimizer stripped from jassair/test_run_14/weights/best.pt, 5.5MB

Validating jassair/test_run_14/weights/best.pt...
Ultralytics 8.3.98 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA A16, 14891MiB)
YOLO11n summary (fused): 100 layers, 2,589,172 parameters, 0 gradients, 6.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:15<00:00,  2.10it/s]


                   all       2000       9002      0.998      0.998      0.995      0.985
             Eichel 10        238        238      0.999          1      0.995      0.991
              Eichel 6        249        249      0.999          1      0.995      0.992
              Eichel 7        241        241      0.999          1      0.995      0.994
              Eichel 8        263        263      0.999          1      0.995      0.993
              Eichel 9        238        238      0.999          1      0.995      0.995
            Eichel Ass        249        249      0.998          1      0.995      0.936
          Eichel Konig        269        269      0.999          1      0.995      0.993
           Eichel Ober        243        243      0.999          1      0.995      0.994
          Eichel Under        231        231      0.999          1      0.995      0.994
               Rose 10        243        243      0.999          1      0.995      0.994
                Rose 

lr/pg0,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,██▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
lr/pg2,█████▇▇▇▆▆▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁
metrics/mAP50(B),▃▆▁▇▇▅█▃▆█████▇███████▇█████████████████
metrics/mAP50-95(B),▂▂▁▄▂▅▄▃▃▆▅▇▇▇▇█████████████████████████
metrics/precision(B),▁▆▇▇▆█▇███▇█████████████████████████████
metrics/recall(B),▁▇▇▇▆██▇▇███████████████████████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ff1381f3cb0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.0380

In [14]:
wandb.finish()

In [15]:
model_path = "jassair/test_run_13/weights/best.pt"

model = YOLO(model_path)

# print("Model loaded:", model.model.yaml['model'])

In [16]:
from PIL import Image, ImageOps
from pathlib import Path
import random

def random_test_image(pad_to_square=True):
    test_path = Path("/exchange/dspro2/jassair/real_life_test/images")
    
    image_files = list(test_path.glob("*.png")) + list(test_path.glob("*.jpg"))
    random_image_path = random.choice(image_files)

    image = Image.open(random_image_path)

    # Convert to RGB if needed
    if image.mode in ("RGBA", "P"):
        image = image.convert("RGB")

    # Resize longest side to 640 while keeping aspect ratio
    width, height = image.size
    scale = 640 / max(width, height)
    new_size = (int(width * scale), int(height * scale))
    image = image.resize(new_size, Image.Resampling.LANCZOS)

    if pad_to_square:
        # Add padding to make it 640x640
        delta_w = 640 - image.size[0]
        delta_h = 640 - image.size[1]
        padding = (delta_w // 2, delta_h // 2, delta_w - delta_w // 2, delta_h - delta_h // 2)
        image = ImageOps.expand(image, padding, fill=(114, 114, 114))  # YOLO-style gray padding

    return image



In [57]:
test_image = random_test_image()

In [58]:
plt.figure(figsize=(8, 8))
plt.imshow(test_image)
plt.axis("off")
plt.show()

<Figure size 800x800 with 1 Axes>

In [59]:
results = model(test_image, conf=0.1)


0: 640x640 (no detections), 14.5ms
Speed: 2.3ms preprocess, 14.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


In [52]:
results = model(test_image, conf=0.01)
results[0].plot()



0: 640x640 1 Schilte Under, 15.4ms
Speed: 2.3ms preprocess, 15.4ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)


array([[[160, 165, 180],
        [160, 165, 180],
        [159, 164, 179],
        ...,
        [172, 181, 195],
        [171, 180, 194],
        [171, 180, 194]],

       [[160, 165, 180],
        [160, 165, 180],
        [159, 164, 179],
        ...,
        [172, 181, 195],
        [171, 180, 194],
        [171, 180, 194]],

       [[159, 164, 179],
        [159, 164, 179],
        [159, 164, 179],
        ...,
        [172, 181, 195],
        [171, 180, 194],
        [171, 180, 194]],

       ...,

       [[ 86,  97, 111],
        [119, 128, 142],
        [145, 152, 167],
        ...,
        [108, 128, 115],
        [108, 128, 115],
        [109, 129, 116]],

       [[ 74,  84,  98],
        [ 60,  69,  83],
        [ 84,  91, 106],
        ...,
        [108, 128, 115],
        [108, 128, 115],
        [108, 128, 115]],

       [[113, 124, 138],
        [ 87,  96, 110],
        [ 60,  68,  82],
        ...,
        [108, 128, 115],
        [107, 127, 114],
        [107, 127, 114]]

In [53]:
# Get the first result's image with annotations
annotated_img = results[0].plot()  # This returns a NumPy array with boxes drawn

# Show with matplotlib
plt.figure(figsize=(10, 10))
plt.imshow(annotated_img)
plt.axis("off")
plt.title("Predictions")
plt.show()

<Figure size 1000x1000 with 1 Axes>